# Planck Quantum SDK - Quick Start

This notebook demonstrates how to use the Planck SDK to run quantum circuits remotely.

**Requirements:**
- A Planck API key (get one at https://planck.plancktechnologies.xyz/qsaas/settings)

---

## 1. Install the SDK

Install directly from GitHub - no need to clone the repository!

In [ ]:
!pip install git+https://github.com/HectorNaaa/Planck-QSaaS.git#subdirectory=sdk/python

## 2. Import and Configure

Replace `YOUR_API_KEY` with your actual Planck API key.

In [ ]:
from planck_sdk import PlanckClient, QuantumCircuit, ExecutionResult

# Option 1: Direct API key
API_KEY = "YOUR_API_KEY"  # Replace with your API key

# Option 2: Use Colab secrets (recommended for security)
# from google.colab import userdata
# API_KEY = userdata.get('PLANCK_API_KEY')

# Initialize the client
client = PlanckClient(
    api_key=API_KEY,
    base_url="https://planck.plancktechnologies.xyz"
)

print("Planck SDK initialized successfully!")

## 3. Run Your First Quantum Circuit

Let's run a simple VQE (Variational Quantum Eigensolver) algorithm with sample data.

In [ ]:
# Sample data for quantum processing
my_data = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]

# Run the quantum circuit
result = client.run(
    data=my_data,
    algorithm="vqe",
    shots=1024
)

print(f"Execution completed!")
print(f"Runtime: {result.runtime_ms:.1f}ms")
print(f"Fidelity: {result.fidelity:.3f}")
print(f"\nMeasurement counts:")
for state, count in sorted(result.counts.items(), key=lambda x: -x[1])[:5]:
    print(f"  |{state}>: {count}")

## 4. Try Different Algorithms

Available algorithms:
- `vqe` - Variational Quantum Eigensolver (optimization)
- `grover` - Grover's Search (database search)
- `qaoa` - QAOA (combinatorial optimization)
- `qft` - Quantum Fourier Transform
- `bell` - Bell State (entanglement)

In [ ]:
# Grover's Search Algorithm
grover_result = client.run(
    data=[0, 1, 0, 1],  # Search for pattern
    algorithm="grover",
    shots=2048
)

print("Grover's Search Results:")
print(f"Fidelity: {grover_result.fidelity:.3f}")
print(f"Top results: {dict(list(sorted(grover_result.counts.items(), key=lambda x: -x[1])[:3]))}")

In [ ]:
# QAOA for optimization
qaoa_result = client.run(
    data=[1.5, 2.3, 0.8, 3.1, 1.9],
    algorithm="qaoa",
    shots=1024,
    error_mitigation="high"
)

print("QAOA Results:")
print(f"Fidelity: {qaoa_result.fidelity:.3f}")
print(f"Runtime: {qaoa_result.runtime_ms:.1f}ms")

## 5. Generate Circuit Without Executing

You can generate the QASM code without running it.

In [ ]:
# Generate circuit only
circuit = client.generate_circuit(
    data=[1, 2, 3, 4],
    algorithm="bell",
    qubits=4
)

print(f"Generated Circuit:")
print(f"  Qubits: {circuit.qubits}")
print(f"  Depth: {circuit.depth}")
print(f"  Gates: {circuit.gate_count}")
print(f"  Recommended shots: {circuit.recommended_shots}")
print(f"\nQASM Code:")
print(circuit.qasm)

## 6. Get ML-Powered Recommendations

The platform uses machine learning to recommend optimal execution parameters.

In [ ]:
# Get ML recommendations
recommendations = client.get_recommendations(
    qubits=8,
    depth=15,
    gate_count=40,
    algorithm="vqe",
    data_size=100
)

print("ML Recommendations:")
print(f"  Recommended shots: {recommendations['recommended_shots']}")
print(f"  Recommended backend: {recommendations['recommended_backend']}")
print(f"  Error mitigation: {recommendations['recommended_error_mitigation']}")
print(f"  Confidence: {recommendations['confidence']:.1%}")

## 7. Ask the AI Assistant

Get help with quantum computing concepts or your executions.

In [ ]:
# Ask the AI assistant
answer = client.ask("What is the difference between VQE and QAOA?")
print(answer)

## 8. Working with Results

Export and analyze your results.

In [ ]:
# Get detailed result info
print(f"Execution ID: {result.execution_id}")
print(f"Algorithm: {result.algorithm}")
print(f"Backend: {result.backend}")
print(f"Shots: {result.shots}")
print(f"Success Rate: {result.success_rate:.1%}")

# Export to JSON
result_json = result.to_json()
print(f"\nJSON export ready ({len(result_json)} bytes)")

---

## Next Steps

- **Get your API key**: https://planck.plancktechnologies.xyz/qsaas/settings
- **Full documentation**: https://github.com/HectorNaaa/Planck-QSaaS/tree/main/sdk/python
- **Platform dashboard**: https://planck.plancktechnologies.xyz/qsaas/dashboard

Happy quantum computing!